In [ ]:
import numpy as np
from google.colab import drive
import sys
from PIL import Image
import os
from os import listdir
import keras

! pip install rasterio 1>/dev/null

from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/sen2dwater/USANNIOMIT')

from dataio.datahandler import datahandler
from dataio.datareader import datareader

from dataio.datahandler import datahandler
from dataio.datareader import datareader
# from models.LSTMv1 import LSTMv1
# from models.BLSTMv1 import BLSTMv1
from models.TDCNNv1 import TDCNNv1

# from utils.plot_dataset import plot_series 

from config import *

import matplotlib.pyplot as plt
import os

%matplotlib inline


# sys.path.append('/content/drive/MyDrive/results-lenovo')

path_to_ndvi_images = '/content/drive/MyDrive/results-lenovo/TDCNNv1/ndvi/res/pr/epoch-14'
path_to_ndwi_images = '/content/drive/MyDrive/results-lenovo/TDCNNv1/ndwi/res/pr/epoch-14'
path_to_nddi_images = '/content/drive/MyDrive/results-lenovo/TDCNNv1/nddi/res/pr/epoch-14'


def compute_NDDI(NDWI,NDVI):
    '''
        Calculates the normalized difference betweeen two indices.

        Inputs:
            - NDWI: NDWI image as computed by normalized_difference method. Must be a numpy
                    array (W,H,B) width W, height H and B bands
            - NDVI: NDVI image as computed by normalized_difference method. Must be a numpy
                    array (W,H,B) width W, height H and B bands
        Output:
            - NDDI: normalized difference of NDWI and NDVI bands 
        ------------------------------------------------------------------------
        The formula applied is the following

                            nd = (NDVI - NDWI)/((NDVI + NDWI) + e)

        e ~= 0.00001 to stabilize division with weak denominator.
    '''
    return (NDVI - NDWI) / (NDVI + NDWI + 0.00001)

#======================================= Loading the dataset ========================================
# dataset_root_path = '/content/drive/MyDrive/sen2dwater/USANNIOMIT/DATASET_2016_2022'
dataset_root_path = '/content/drive/MyDrive/SEN2DWATER'

dh       = datahandler(dataset_root_path)
keys     = list(dh.paths.keys())
t_len    = len(dh.paths[keys[0]])

print('{:=^100}'.format(' Loading the dataset '))
print('\t -{:<50s} {}'.format('Number of GeoLocation', len(keys)))
print('\t -{:<50s} {}'.format('Number of Images per GeoLocation', t_len))

#========================================== Split dataset ===========================================
train_set, val_set = dh.split(SPLIT_FACTOR)
print('{:=^100}'.format(' Splitting the dataset '))
print('\t -{:<50s} {}'.format('Number of GeoLocation (training)', len(train_set.keys())))
print('\t -{:<50s} {}'.format('Number of GeoLocation (validation)', len(val_set.keys())))

tdcnnv1 = TDCNNv1(shape=(T_LEN, PATCH_WIDTH, PATCH_HEIGHT, BANDS))
print('{:=^100}'.format(' Building the Model '))
print(tdcnnv1.model.summary())
#========================================== Train model =============================================
print('{:=^100}'.format(' Training the Model '))
# history = tdcnnv1.train(train_set, val_set, normalize = True)

tdcnnv1.generateData(train_set, val_set)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
======================================= Loading the dataset ========================================
	 -Number of GeoLocation                              5264
	 -Number of Images per GeoLocation                   39
====================================== Splitting the dataset =======================================
	 -Number of GeoLocation (training)                   4212
	 -Number of GeoLocation (validation)                 1052
======================================== Building the Model ========================================
Model: "TDCNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6, None, None, 1  0         
                             )]                                  
                                                       

In [ ]:
ndvi_list = []
ndwi_list = []
computed_nddi_list = []
gt_nddi_list = []


def load(path):
  out = []
  for image in os.listdir(path):
    np_image = np.asarray(Image.open(path + "/" + image))
    out.append(np_image)
  return out

ndvi_list = load(path_to_ndvi_images)
ndwi_list = load(path_to_ndwi_images)
gt_nddi_list = load(path_to_nddi_images)

print(len(ndvi_list))
print(len(ndwi_list))

for i in range(100):
  # compute_NDDI(NDWI,NDVI)
  # for each predicted ndwi and ndvi image, calculate the nddi value
  computed_nddi = compute_NDDI(ndwi_list[i], ndvi_list[i])
  # store in array
  computed_nddi_list.append(computed_nddi)


# print(computed_nddi_list[0] - gt_nddi_list[0])
# print(computed_nddi_list[0])
# print(gt_nddi_list[0])



100
100
[[[2.55319122e+00 1.69999660e+01 2.03333164e+01 0.00000000e+00]
  [2.83333300e+00 1.16666472e+01 1.37777701e+01 0.00000000e+00]
  [2.90588201e+00 2.99999864e+00 1.74285590e+01 0.00000000e+00]
  ...
  [1.88721790e+00 5.61983448e-01 9.99999959e-01 0.00000000e+00]
  [2.03999984e+00 3.89998700e+01 9.36254943e-01 0.00000000e+00]
  [4.05405378e-02 6.72413764e-01 1.08888884e+00 0.00000000e+00]]

 [[2.09433943e+00 5.59633002e-01 1.96078424e-02 0.00000000e+00]
  [3.81679360e-02 4.03331989e+01 9.43319800e-01 0.00000000e+00]
  [2.32710259e+00 9.59999040e+00 4.77999044e+01 0.00000000e+00]
  ...
  [1.92982439e-01 2.02438975e+00 8.89763744e-01 0.00000000e+00]
  [1.40845061e-01 7.49999531e+00 9.25311165e-01 0.00000000e+00]
  [1.16564410e-01 7.46724858e-01 1.16267937e+00 0.00000000e+00]]

 [[6.57894694e-02 6.47058796e-01 1.06222218e+00 0.00000000e+00]
  [2.19266035e+00 4.29718858e-01 6.09998475e+01 0.00000000e+00]
  [2.88235260e+00 3.52631393e+00 1.74285590e+01 0.00000000e+00]
  ...
  [2.91666

In [ ]:
path_to_gt_ndvi_images = '/content/drive/MyDrive/results-lenovo/TDCNNv1/ndvi/res/gt/epoch-36'

ndvi_model = keras.models.load_model('/content/drive/MyDrive/results-lenovo/ndvi_model.h5')
print(type(ndvi_model))
ndvi_model.predict()
# his = ndvi_model.fit()
# print(his.params)

# for image in os.listdir(path_to_gt_ndvi_images):
#   np_image = np.asarray(Image.open(path_to_gt_ndvi_images + "/" + image))
#   ndvi_model.predict(np_image)



<class 'keras.engine.functional.Functional'>


AttributeError: ignored

In [ ]:
from IPython.core.interactiveshell import re
ndwiimg = "/content/pt-1.tiff"
ndviimg = "/content/pt-0.tiff"

ndwi_image = np.asarray(Image.open(ndwiimg))
ndvi_image = np.asarray(Image.open(ndviimg))

res = compute_NDDI(ndwi_image, ndvi_image)
print(res)
print(gt_nddi_list[0])